<h2 style="color:blue; font-size:23pt;">
CNN-Synth: Generalised, minimally invasive cancer detection
</h2>

In [ ]:
%matplotlib inline

import time
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import warnings

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
from   sklearn.preprocessing import StandardScaler, RobustScaler
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.metrics import RocCurveDisplay, auc, roc_curve, precision_recall_curve, average_precision_score, accuracy_score, confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers, models, losses
import tensorflow as tf
import shap

from sklearn import metrics
from sklearn.metrics import classification_report

np.set_printoptions(suppress=True)
np.set_printoptions(precision=4)
plt_style = 'seaborn-talk'


<h2 style="color:blue; font-size:20pt;">
Load and preprocess the proteomic dataset
</h2>

In [ ]:
print("Reading Data Matrix")
data_df = pd.read_csv('Data/.csv') #path to data file
filtered_NPX_data_array = data_df.to_numpy()
print(filtered_NPX_data_array.shape)
#print(data_df.head())

sample_IDs = data_df["SampleID"].tolist()
assay_92_names = data_df.columns.values.tolist()
assay_92_names = assay_92_names[1:]

print([len(sample_IDs), len(assay_92_names)])
NPX_real_data_array = filtered_NPX_data_array[:,1:]
NPX_real_data_array = np.asarray(NPX_real_data_array).astype(np.float32)


# --- Filter and reorder protein features ---

# Your protein list
with open('Data/92_ProteinList_gene_names.txt', 'r') as file:
    selected_proteins = [line.strip() for line in file if line.strip()]

# Normalize function to match names
def normalize(name):
    return name.upper().replace("-", "").replace("_", "")

# Map original column names to normalized forms
normalized_assay_names = [normalize(name) for name in assay_92_names]
assay_map = {normalize(name): name for name in assay_92_names}

# Filter and reorder
matched_proteins = []
missing_proteins = []

for p in selected_proteins:
    norm_p = normalize(p)
    if norm_p in assay_map:
        matched_proteins.append(assay_map[norm_p])
    else:
        missing_proteins.append(p)

print(f"Matched: {len(matched_proteins)}, Missing: {missing_proteins}")

if len(missing_proteins) > 1:
    warnings.warn(f"Multiple unmatched proteins found: {missing_proteins}")
    raise ValueError(f"Ensure that you are working with same set of 92 proteins, in same order!")

<h2 style="color:blue; font-size:20pt;">
Load case (should be 1), control (should be 0) information
</h2>

In [ ]:
gt_pat_type_data = pd.read_csv('Data/.csv') #path to phenotype file, rows = sampleIDs, columns = Case (1) or Control (0)  
gt_pat_type_data = gt_pat_type_data.to_numpy()
print(gt_pat_type_data.shape)
gt_pat_type = gt_pat_type_data[:,1]

gt_pat_type = np.where(gt_pat_type == 2, 0, gt_pat_type)
gt_pat_type_array = gt_pat_type.reshape(gt_pat_type.shape[0], 1)
gt_pat_type_array = np.asarray(gt_pat_type_array).astype(np.float32)

gt_pat_type = list(gt_pat_type)
print([gt_pat_type_array.shape, NPX_real_data_array.shape])
print('Number of Cancer Patients: ' + str(gt_pat_type.count(1)))
print('Number of Healthy Patients: ' + str(gt_pat_type.count(0)))

NPX_real_data_array = np.expand_dims(NPX_real_data_array, axis=1)

<h2 style="color:blue; font-size:20pt;">
Define and load the pre-trained deep learning model
</h2>

In [ ]:
latent_dim = 10

classifier_inputs = keras.Input(shape=(1,92))
x = layers.Conv1D(64, 1, strides=1, activation="relu", padding="same",kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001))(classifier_inputs)
x = layers.Conv1D(64, 1, strides=1, activation="relu", padding="same",kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation="relu", kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(32, activation="relu", kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(10, activation="relu", kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
#x = layers.Dense(128, activation="relu")(x)
#x = layers.Dense(10, activation="tanh", kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001))(x)
classifier_output = layers.Dense(1, activation="sigmoid", name="output", kernel_initializer='he_uniform')(x)
nn_classifier = keras.Model(classifier_inputs, classifier_output, name="encoder")

nn_classifier.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

initial_weights_path = "Output/CNN-Synth.weights.h5" #path to pre-trained weights
nn_classifier.summary()
nn_classifier.load_weights(initial_weights_path)


<h2 style="color:blue; font-size:20pt;">
Evaluate model: accuracy, AUC, ROC curve
</h2>

In [ ]:
X_test = NPX_real_data_array
y_pred = nn_classifier.predict(X_test)

# Metrics
fpr, tpr, _ = roc_curve(gt_pat_type_array, y_pred)
roc_auc = auc(fpr, tpr)

# Plot ROC
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
roc_auc = auc(fpr, tpr)
precision, recall, thresholds = precision_recall_curve(gt_pat_type_array, y_pred)
#print([precision, recall])
fscore = (2 * precision * recall) / (precision + recall)
AP = average_precision_score(gt_pat_type_array, y_pred)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%.3f, F1-score=%.3f' % (thresholds[ix], fscore[ix]))
y_pred = np.where(y_pred > thresholds[ix], 1, 0)
confusion_matrix = confusion_matrix(gt_pat_type_array, y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])
cm_display.plot()
plt.show()

target_names = ['Control', 'Case']
print(classification_report(gt_pat_type_array, y_pred, target_names=target_names))

<h2 style="color:blue; font-size:20pt;">
Generate SHAP values and plot feature importance
</h2>

In [ ]:
NpX_train, NpX_test, Npy_train, Npy_test = train_test_split(NPX_real_data_array, gt_pat_type_array, test_size=0.8, random_state=42)
print([NpX_train.shape, NpX_test.shape])

In [ ]:
NPX_real_data_array_ap = np.expand_dims(NPX_real_data_array, axis=1)
NpX_test_ap = np.expand_dims(NpX_test, axis=1)
NpX_train_ap = np.expand_dims(NpX_train, axis=1)

In [ ]:
NPX_real_data_pd = pd.DataFrame(NPX_real_data_array_ap[:,0,:], columns=assay_92_names)
NpX_test_data_pd = pd.DataFrame(NpX_test_ap[:,0,:], columns=assay_92_names)
NpX_train_data_pd = pd.DataFrame(NpX_train_ap[:,0,:], columns=assay_92_names)

explainer = shap.DeepExplainer(nn_classifier, NPX_real_data_array_ap[:, :, :])
shap_values = explainer.shap_values(NPX_real_data_array_ap)

print(explainer)
Base_value = explainer.expected_value
print(Base_value)

print([shap_values.shape, NPX_real_data_array.shape])
shap_values_V_reshaped = shap_values.reshape((175, 92))

print([shap_values_V_reshaped.shape, NPX_real_data_array_ap.shape])

#shap.summary_plot(shap_values_V_reshaped, NPX_real_data_pd)
shap.summary_plot(shap_values_V_reshaped, NPX_real_data_pd, plot_type='bar')
shap_values_cnn = shap_values_V_reshaped